In [1]:
import pandas as pd
from sqlite_utils import Database

In [2]:
db = Database("mybinder_archive.db")

In [3]:
# get table names
db.table_names()

['mybinderlaunch', 'repo']

In [4]:
launch_table = 'mybinderlaunch'

In [5]:
# db[launch_table].columns
db[launch_table].columns_dict

{'timestamp': float,
 'version': int,
 'origin': str,
 'provider': str,
 'spec': str,
 'org': str,
 'ref': str,
 'resolved_ref': str,
 'repo_url': str}

In [6]:
# NOTE: this notebook wont have same output (nbs_same_output)
query = f'SELECT * FROM {launch_table} ORDER BY RANDOM() LIMIT 100'
pd.read_sql_query(query, db.conn)

,timestamp,version,origin,provider,spec,org,ref,resolved_ref,repo_url
0,2018-12-04 08:53:00+00:00,1,mybinder.org,GitHub,jupyterlab/jupyterlab-demo/master,jupyterlab,master,,https://github.com/jupyterlab/jupyterlab-demo
1,2019-02-18 00:35:00+00:00,1,mybinder.org,GitHub,JohnLaTwC/Shared/master,JohnLaTwC,master,,https://github.com/johnlatwc/shared
2,2019-11-20 18:15:00+00:00,3,gke.mybinder.org,GitHub,mwouts/itables/master,mwouts,master,,https://github.com/mwouts/itables
3,2019-12-21 10:19:00+00:00,3,notebooks.gesis.org,GitHub,ipython/ipython-in-depth/master,ipython,master,,https://github.com/ipython/ipython-in-depth
4,2020-04-28 04:47:00+00:00,3,gke.mybinder.org,GitHub,clumdee/quantum_well_solver/master,clumdee,master,,https://github.com/clumdee/quantum_well_solver
...,...,...,...,...,...,...,...,...,...
95,2019-11-03 15:09:00+00:00,3,notebooks.gesis.org,GitHub,ipython/ipython-in-depth/master,ipython,master,,https://github.com/ipython/ipython-in-depth
96,2019-10-10 13:14:00+00:00,3,gke.mybinder.org,GitHub,jupyterlab/jupyterlab-demo/try.jupyter.org,jupyterlab,try.jupyter.org,,https://github.com/jupyterlab/jupyterlab-demo
97,2020-05-14 01:48:00+00:00,3,gesis.mybinder.org,GitHub,ipython/ipython-in-depth/master,ipython,master,,https://github.com/ipython/ipython-in-depth
98,2019-08-22 08:20:00+00:00,3,gke.mybinder.org,GitHub,ipython/ipython-in-depth/master,ipython,master,,https://github.com/ipython/ipython-in-depth


### Total number of launches

In [7]:
db[launch_table].count

9624058

First timestamp

In [8]:
launches = db[launch_table].rows_where(order_by="timestamp")
first_launch_timestamp = next(launches)["timestamp"]
del launches

first_launch_timestamp

'2018-11-03 00:00:00+00:00'

Last timestamp

In [9]:
launches = db[launch_table].rows_where(order_by="timestamp desc")
last_launch_timestamp = next(launches)["timestamp"]
del launches

last_launch_timestamp

'2020-06-30 23:59:00+00:00'

### Launches per origin

In [10]:
query = f'SELECT origin, count(origin) FROM {launch_table} GROUP BY "origin";'
launches_per_origin = pd.read_sql_query(query, db.conn)
launches_per_origin

,origin,count(origin)
0,binder.mybinder.ovh,556
1,binder.mybinder.turing.ac.uk,129
2,gesis.mybinder.org,681920
3,gke.mybinder.org,4754321
4,gke.mybinder.org:443,1
5,mybinder.org,2741924
6,notebooks.gesis.org,668369
7,ovh.mybinder.org,535332
8,turing.mybinder.org,241506


In [11]:
launches_per_origin_dict = {"gke": 0, "gesis": 0, "turing": 0, "ovh": 0}
for index, row in launches_per_origin.iterrows():
    if row["origin"] == "mybinder.org":
        launches_per_origin_dict["gke"] += row["count(origin)"]
    for origin in launches_per_origin_dict.keys():
        if origin in row["origin"]:
            # print(origin, row["origin"])
            launches_per_origin_dict[origin] += row["count(origin)"]

assert sum(launches_per_origin_dict.values()) == db[launch_table].count

pd.DataFrame.from_dict(
    launches_per_origin_dict, 
    orient="index", 
    columns=["launches"]
).sort_values(by=["launches"], ascending=False)

,launches
gke,7496246
gesis,1350289
ovh,535888
turing,241635


### Launches per provider

In [12]:
query = f'SELECT provider, count(provider) FROM {launch_table} GROUP BY "provider";'
launches_per_provider = pd.read_sql_query(query, db.conn).sort_values(by=["count(provider)"], ascending=False)

launches_per_provider

,provider,count(provider)
4,GitHub,9388681
2,Gist,131204
3,Git,63858
5,GitLab,38556
7,Zenodo,943
6,Hydroshare,383
1,Figshare,306
0,Dataverse,127


In [13]:
# percentage of first 2 providers
(sum(launches_per_provider.head(2)["count(provider)"])*100)/sum(launches_per_provider["count(provider)"])

98.91757718002115

### Organisations/Users in repos

In [14]:
# number of different orgs
query = "select count(distinct org) from mybinderlaunch;"
db.conn.execute(query).fetchone()[0]

33514